<center><h1>2. Vainilla CNNs</h1></center>

In [ ]:
# Necessary modules
import pandas as pd
import numpy  as np

from keras.models     import Sequential
from keras.layers     import Dense, Dropout, Activation, Flatten
from keras.layers     import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
# Get the data
data = pd.read_csv(filepath_or_buffer="../data/face_image_project/fold_0_data.txt", sep="\t")
data = data.sample(frac=1).reset_index(drop=True)
data.head()

In [ ]:
data = data[["user_id","original_image","face_id","gender"]]
data.head()

In [ ]:
data.loc[data.gender != "m", "gender"] = 0
data.loc[data.gender == "m", "gender"] = 1
data.head()

In [ ]:
path_template = "../data/face_image_project/aligned/%s/landmark_aligned_face.%d.%s"
data["file_path"] = data[["user_id","face_id","original_image"]].apply(lambda x:  
                                                                   path_template % (x[0],x[1],x[2]),
                                                                   axis=1)
data.head()

In [ ]:
data = data[["file_path","gender"]]
data.head()

In [ ]:
data["gender"].value_counts()

Occurences of each gender seem pretty balanced. Looks good to me

In [ ]:
Y      = data["gender"]
X_path = data["file_path"]

In [ ]:
X_path[0]

In [ ]:
tempo = X_path[1:3]
tempo.apply(lambda x: img_to_array( load_img(x) ) ) #DOES IT NEED A RESHAPE

In [ ]:
import scipy.misc

i_width  = 227
i_height = 227

#scipy.misc.imresize(original_image, (i_height, i_width))


def train_generator(data):
    while True:
        start, end = 0, 32
        while end < len(data):
            data = data.sample(frac=1).reset_index(drop=True)
            sample  = data[start:end]

            X = pd.DataFrame(sample["file_path"].apply(lambda x:  img_to_array( scipy.misc.imresize(load_img(x), (i_height, i_width) ) ) ))
            
            X = X["file_path"].apply(lambda x: x.reshape((1,)+ x.shape))
            X = np.vstack(X)
            #print X[0].shape
            #print X.shape
            #
            Y = sample["gender"].as_matrix()
            #print Y.as_matrix()
            #print X.shape
            #print X
            yield (X, Y)
            start += 32
            end += 32

In [ ]:
gen = train_generator(data)
a = gen.next()
a[0].shape

In [ ]:
# Building a vainilla CNN
vainilla_cnn = Sequential()
vainilla_cnn.add(Convolution2D(32, 3, 3, input_shape=(227,227,3)))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(MaxPooling2D(pool_size=(2, 2)))
vainilla_cnn.add(Convolution2D(64, 3, 3))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(Convolution2D(64, 3, 3))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(MaxPooling2D(pool_size=(2, 2)))
vainilla_cnn.add(Flatten())
vainilla_cnn.add(Dense(output_dim=200, input_dim=500))
vainilla_cnn.add(BatchNormalization())
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dense(output_dim=100, input_dim=200))
vainilla_cnn.add(BatchNormalization())
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dense(output_dim=1))
vainilla_cnn.add(Activation("sigmoid"))

In [ ]:
# Compile
vainilla_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train
vainilla_cnn.fit_generator(train_generator(data), samples_per_epoch=64, nb_epoch=200)

In [ ]:
!nvidia-smi

In [ ]:
def rowToImgPath(row):
    userid = row["user_id"]
    faceid = row["face_id"]
    original_img = row["original_image"]
    imgPath = "../data/face_image_project/aligned/%s/landmark_aligned_face.%d.%s" % (userid, faceid, original_img)
    return imgPath

def loadXY(data):
    """ BE INCREDIBLY CAREFUL... The data is large - you could run out of memory and start thrashing.
    It is highly advised to load a small subset at a time (100-200 rows)
    """ 
    X = np.zeros((len(data), 816, 816, 3))
    Y = data["gender"]
    for i in range(len(data)):
        row = data.iloc[i]
        img = load_img(rowToImgPath(row))
        imgArray = img_to_array(img)
        X[i] = imgArray
    return X, Y

def generateXY(data, size=5):
    """ From Keras fit_generator API: The generator is expected to loop over its data indefinitely. """
    while True:
        start, end = 0, size
        while end <= len(data):
            yield loadXY(data[start: end])
            start, end =  start + size, end + size

            
def loadXY2(data):
    """ BE INCREDIBLY CAREFUL... The data is large - you could run out of memory and start thrashing.
    It is highly advised to load a small subset at a time (100-200 rows)
    """ 
    X = np.zeros((len(data), 816, 816, 3))
    Y = data["gender"]
    for i in range(len(data)):
        row = data.iloc[i]
        img = load_img(rowToImgPath(row))
        imgArray = img_to_array(img)
        X[i] = imgArray
    return X, Y

def generateXY2(data, size=32):
    """ From Keras fit_generator API: The generator is expected to loop over its data indefinitely. """
    while True:
        start, end = 0, size
        while end <= len(data):
            yield loadXY(data[start: end])
            start, end =  start + size, end + size
